In [ ]:
# 📦 Step 1: Import e path
import sys
import os
import random
import matplotlib.pyplot as plt
from PIL import Image

sys.path.append("../src")  # per PyCharm
# oppure "/content/src" se su Colab

from src.dataset_loader import get_dataloader

# 🧹 Step 2: Prepara il dataset (se non è già stato fatto)
prepared_path = "datasets/dataset_prepared/train"  # PyCharm
# oppure "/content/dataset_prepared/train" su Colab

if not os.path.exists(prepared_path):
    print("⚠️ Dataset non ancora preparato. Avvio preprocessing...")
    from src.prepare_dataset import prepare_dataset

    prepare_dataset()
else:
    print("✅ Dataset già presente. Salto preprocessing.")

# 📥 Step 3: Carica il dataloader (solo train per ora)
dataset, dataloader = get_dataloader(
    data_dir=prepared_path,
    batch_size=32
)

print(f"✅ Dataset caricato correttamente! Immagini: {len(dataset)}")

# 🖼️ Step 4: Visualizza un'immagine a caso
sample_path = os.path.join(prepared_path, "IDC")
img_name = random.choice(os.listdir(sample_path))
img = Image.open(os.path.join(sample_path, img_name))

plt.imshow(img)
plt.title("Esempio IDC (cancerosa)")
plt.axis("off")
plt.show()
